< [The PCRaster Dynamic Modelling Framework](PCRasterDynamicModellingFramework.ipynb) | [Contents](Contents.ipynb) | [Building the dynamic model](STREAM_Dynamic.ipynb) >

# 3. Prepare the `initial` section of the model
A dynamic model with the PCRaster Python framework has an `initial` and a `dynamic` section. As discussed before the initial section contains code that needs to be run only once to initialise the model. It should not be iterated in time.

## 3.1 Read static maps
We need to initialise the STREAM model by reading the following static maps in the `initial` section:
```Python
landuse.map       # land-use map
soil.map          # soil map
metstat.map       # map with meteorological stations
dem.map           # Digital elevation model
mask.map          # clone map
```

Add the lines to the `initial` section of the script to read the static maps and run the script to check if there are no errors. Note that we use `landuse.map` and `soil.map` only in the `initial` section to create maps with lookup tables. We're also not using `dem.map` the `dynamic` section. All other maps are used in both the `initial`and `dynamic` section. So think about how to create local and global variables.

In [ ]:
from pcraster import *
from pcraster.framework import *

class RunoffModel(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
        # Add here the static maps that we need to read from disk
        landuse = self.readmap("./Data/landuse")
        soil = self.readmap("./Data/soil")
        
        
        # Add here the lookup tables
        self.InterceptionThreshold = lookupscalar("./Data/d.tbl",landuse)
        self.report(self.InterceptionThreshold,"d")
    
    def dynamic(self):
        Precipitation = self.readmap("./Data/pr") 
        
myModel = RunoffModel("./Data/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=10, firstTimestep=1)
dynModelFw.run()

## 3.2 Define parameters (constants)
We also need to set the model parameters (contstants) in the initial section. These constants will be used in the `dynamic` section so we need to make them global. They are added below with in the comments their meaning. Later in the `dynamic` section it will become clearer how these constants will be used.

In [ ]:
from pcraster import *
from pcraster.framework import *

class RunoffModel(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
        # Add here the static maps that we need to read from disk
        landuse = self.readmap("./Data/landuse")
        soil = self.readmap("./Data/soil")
        self.metstat = self.readmap("./Data/metstat")
        DEM = self.readmap("./Data/dem")
        self.mask = self.readmap("./Data/mask")
        
        # Add here the constants as global variables
        self.Ku = scalar(1.5)                  # Recession constant of flow from unsaturated to saturated zone
        self.surface = scalar(0.01)            # surface of a gridcell (km2)
        self.ConvConst = scalar(0.00001157407) # From mm/10 days to m3/s: *1/1000 * 1/10 *
                                               # 1/24 * 1/3600 * 100^2
        self.rtq = scalar(1.2)                 # Recession constant quick flow
        self.rts = scalar(5.3)                 # Recession constant slow flow
        self.Su = scalar(50.0)                 # storage unsaturated zone
        self.Ssmax = scalar(0.0)               # parameter controlling the groundwater flow to the river
        self.Ss = scalar(50.0)                 # storage saturated zone
        
        
        # Add here the lookup tables
        self.InterceptionThreshold = lookupscalar("./Data/d.tbl",landuse)
        self.report(self.InterceptionThreshold,"d")
    
    def dynamic(self):
        Precipitation = self.readmap("./Data/pr") 
        
myModel = RunoffModel("./Data/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=10, firstTimestep=1)
dynModelFw.run()

## 3.3 Create inputs from lookup tables
Now we add the other parameters for which we use lookup tables.

For the soil map these are:
* Maximum storage of the unsaturated zone `SuMax` (mm)
* Quick flow coefficient (-)

We use hypothetical values and soil classes.

For the land-use map these are:
* The separation coefficient (-)
* Maximum capilary rise (mm)

In [ ]:
from pcraster import *
from pcraster.framework import *

class RunoffModel(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
        # Add here the static maps that we need to read from disk
        landuse = self.readmap("./Data/landuse")
        soil = self.readmap("./Data/soil")
        self.metstat = self.readmap("./Data/metstat")
        DEM = self.readmap("./Data/dem")
        self.mask = self.readmap("./Data/mask")
        
        # Add here the constants as global variables
        self.Ku = scalar(1.5)                  # Recession constant of flow from unsaturated to saturated zone
        self.surface = scalar(0.01)            # surface of a gridcell (km2)
        self.ConvConst = scalar(0.00001157407) # From mm/10 days to m3/s: *1/1000 * 1/10 *
                                               # 1/24 * 1/3600 * 100^2
        self.rtq = scalar(1.2)                 # Recession constant quick flow
        self.rts = scalar(5.3)                 # Recession constant slow flow
        self.Su = scalar(50.0)                 # storage unsaturated zone
        self.Ssmax = scalar(0.0)               # parameter controlling the groundwater flow to the river
        self.Ss = scalar(50.0)                 # storage saturated zone
        
        
        # Add here the lookup tables
        self.InterceptionThreshold = lookupscalar("./Data/d.tbl",landuse)   # interception threshold (mm)
        self.report(self.InterceptionThreshold,"./Data/d")
        
        self.SuMax = lookupscalar("./Data/smax.tbl",soil)                   # maximum storage unsaturated zone (mm)
        self.report(self.SuMax,"./Data/SuMax")
        
        self.SeparationCoefficient = lookupscalar("./Data/cr.tbl",landuse)  # separation coefficient (-)
        self.report(self.SeparationCoefficient,"./Data/cr")
        
        self.QuickFlowCoefficient = lookupscalar("./Data/qc.tbl",soil)      # Quick flow coefficient (-)
        self.report(self.QuickFlowCoefficient,"./Data/Qc")
            
        self.MaxCapRise = lookupscalar("./Data/cp.tbl",landuse)             # potential capillary rise (mm)
        self.report(self.MaxCapRise,"./Data/cmax")
      
    
    def dynamic(self):
        Precipitation = self.readmap("./Data/pr") 
        
myModel = RunoffModel("./Data/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=10, firstTimestep=1)
dynModelFw.run()

## 3.4 Create flow direction map
We also create the flow direction map in the `initial` section. Add the code to line 46 and run the script.

In [ ]:
from pcraster import *
from pcraster.framework import *

class RunoffModel(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
        # Add here the static maps that we need to read from disk
        landuse = self.readmap("./Data/landuse")
        soil = self.readmap("./Data/soil")
        self.metstat = self.readmap("./Data/metstat")
        DEM = self.readmap("./Data/dem")
        self.mask = self.readmap("./Data/mask")
        
        # Add here the constants as global variables
        self.Ku = scalar(1.5)                  # Recession constant of flow from unsaturated to saturated zone
        self.surface = scalar(0.01)            # surface of a gridcell (km2)
        self.ConvConst = scalar(0.00001157407) # From mm/10 days to m3/s: *1/1000 * 1/10 *
                                               # 1/24 * 1/3600 * 100^2
        self.rtq = scalar(1.2)                 # Recession constant quick flow
        self.rts = scalar(5.3)                 # Recession constant slow flow
        self.Su = scalar(50.0)                 # storage unsaturated zone
        self.Ssmax = scalar(0.0)               # parameter controlling the groundwater flow to the river
        self.Ss = scalar(50.0)                 # storage saturated zone
        
        
        # Add here the lookup tables
        self.InterceptionThreshold = lookupscalar("./Data/d.tbl",landuse)   # interception threshold (mm)
        self.report(self.InterceptionThreshold,"./Data/d")
        
        self.SuMax = lookupscalar("./Data/smax.tbl",soil)                   # maximum storage unsaturated zone (mm)
        self.report(self.SuMax,"./Data/SuMax")
        
        self.SeparationCoefficient = lookupscalar("./Data/cr.tbl",landuse)  # separation coefficient (-)
        self.report(self.SeparationCoefficient,"./Data/cr")
        
        self.QuickFlowCoefficient = lookupscalar("./Data/qc.tbl",soil)      # Quick flow coefficient (-)
        self.report(self.QuickFlowCoefficient,"./Data/Qc")
            
        self.MaxCapRise = lookupscalar("./Data/cp.tbl",landuse)             # potential capillary rise (mm)
        self.report(self.MaxCapRise,"./Data/cmax")
        
        # Add here the calculation of the flow direction map
   
    
    def dynamic(self):
        Precipitation = self.readmap("./Data/pr") 
        
myModel = RunoffModel("./Data/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=10, firstTimestep=1)
dynModelFw.run()

Now it takes more time to run the model. But the good thing is that it needs to generate the flow direction map only once. In the future you might even just want to read the flow direction map from disk instead of calculating it every time you run the model.

The flow direction map needs to be adjusted to the mask and still be hydrologically correct. We use the [lddmask operation](https://pcraster.geo.uu.nl/pcraster/4.3.0/documentation/pcraster_manual/sphinx/op_lddmask.html).

The code has been added below and the result is saved to disk.

In [ ]:
from pcraster import *
from pcraster.framework import *

class RunoffModel(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
        # Add here the static maps that we need to read from disk
        landuse = self.readmap("./Data/landuse")
        soil = self.readmap("./Data/soil")
        self.metstat = self.readmap("./Data/metstat")
        DEM = self.readmap("./Data/dem")
        self.mask = self.readmap("./Data/mask")
        
        # Add here the constants as global variables
        self.Ku = scalar(1.5)                  # Recession constant of flow from unsaturated to saturated zone
        self.surface = scalar(0.01)            # surface of a gridcell (km2)
        self.ConvConst = scalar(0.00001157407) # From mm/10 days to m3/s: *1/1000 * 1/10 *
                                               # 1/24 * 1/3600 * 100^2
        self.rtq = scalar(1.2)                 # Recession constant quick flow
        self.rts = scalar(5.3)                 # Recession constant slow flow
        self.Su = scalar(50.0)                 # storage unsaturated zone
        self.Ssmax = scalar(0.0)               # parameter controlling the groundwater flow to the river
        self.Ss = scalar(50.0)                 # storage saturated zone
        
        
        # Add here the lookup tables
        self.InterceptionThreshold = lookupscalar("./Data/d.tbl",landuse)   # interception threshold (mm)
        self.report(self.InterceptionThreshold,"./Data/d")
        
        self.SuMax = lookupscalar("./Data/smax.tbl",soil)                   # maximum storage unsaturated zone (mm)
        self.report(self.SuMax,"./Data/SuMax")
        
        self.SeparationCoefficient = lookupscalar("./Data/cr.tbl",landuse)  # separation coefficient (-)
        self.report(self.SeparationCoefficient,"./Data/cr")
        
        self.QuickFlowCoefficient = lookupscalar("./Data/qc.tbl",soil)      # Quick flow coefficient (-)
        self.report(self.QuickFlowCoefficient,"./Data/Qc")
            
        self.MaxCapRise = lookupscalar("./Data/cp.tbl",landuse)             # potential capillary rise (mm)
        self.report(self.MaxCapRise,"./Data/cmax")
        
        # Add here the calculation of the flow direction map
        self.flowdirection = lddcreate(DEM,1e31,1e31,1e31,1e31)
        self.flowdirection = lddmask(self.flowdirection,self.mask)
        self.report(self.flowdirection,"./Data/ldd")
 
    
    def dynamic(self):
        Precipitation = self.readmap("./Data/pr") 
        
myModel = RunoffModel("./Data/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=10, firstTimestep=1)
dynModelFw.run()

Now we can proceed with building the dynamic model in the next section.

< [The PCRaster Dynamic Modelling Framework](PCRasterDynamicModellingFramework.ipynb) | [Contents](Contents.ipynb) | [Building the dynamic model](STREAM_Dynamic.ipynb) >